In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('./train_off.csv')

In [5]:
def remove_single_clickout_actions(df):
  print('Initial size: ' + str(df.shape[0]))
  n_action_session = df.groupby('session_id').size().reset_index(name='n_actions')
  #print(n_action_session.head())
  df = (df.merge(n_action_session, left_on='session_id', right_on='session_id', how="left"))
  #print(df.head())
  # df['step_max'] = df.groupby(['session_id'])['step'].transform(max)
  # print(df.head())
  df = df.drop(df[(df["action_type"] == "clickout item") & (df['n_actions'] == 1)].index)
  print('Final size: ' + str(df.shape[0]))
  del df['n_actions']
  return df

def remove_nonitem_actions(df):
  df = df.drop(df[(df['action_type'] != 'interaction item image') & (df['action_type'] != 'interaction item deals') & (df['action_type'] != 'clickout item') & (df['action_type'] != 'search for item')].index)
  return df

def get_training_input(df_train):
  clickout_dict = {}
  impression_dict = {}
  session_dict = {}

  df_train['step_max'] = df_train[df_train['action_type'] == 'clickout item'].groupby(['session_id'])['step'].transform(max)
  df_train['result'] = df_train[df_train['step'] == df_train['step_max']].apply(lambda x: get_clickout_data(x, clickout_dict, impression_dict), axis = 1)
  #df_train = df_train.drop(df_train.index[math.isnan(df_train['step_max'])])
  df_train_corpus = df_train.groupby('session_id').apply(lambda x: get_list_session_interactions(x, session_dict)).reset_index(name = 'hotel_list')
  df_train = df_train.drop(df_train.index[(df_train['step'] == df_train['step_max']) & (df_train["action_type"] == "clickout item")])

  train_corpus = list(session_dict.values())
  
  #for hotel_session in df_train_corpus.hotel_list.values:
  #  train_corpus.append(hotel_session.split(' '))

  return session_dict, clickout_dict, impression_dict, train_corpus

def get_clickout_data(action, clickout_dict, impression_dict):
  clickout_dict[action.session_id] = action.reference
  impression_dict[action.session_id] = action.impressions.split('|')
  return action.reference

def get_list_session_interactions(group, session_dict):
  session_dict[group.session_id.values[0]] = list(group.reference.values)

  return " ".join(list(group.reference.values))

In [6]:
df_train = remove_single_clickout_actions(df_train)
df_train = remove_nonitem_actions(df_train)

Initial size: 15932992
Final size: 15755447


In [7]:
session_dict, clickout_dict, impression_dict, train_corpus = get_training_input(df_train)

In [8]:
len(session_dict)

702783

In [21]:
count_sessions = 0
first_count = 0
first_2_count = 0
first_3_count = 0
first_4_count = 0
first_5_count = 0

for k, v in session_dict.items():
    if k in clickout_dict:
        count_sessions = count_sessions + 1
        if clickout_dict[k] == list(set(v[::-1]))[0]:
            first_count = first_count + 1
        if clickout_dict[k] in list(set(v[::-1]))[:2]:
            first_2_count = first_2_count + 1
        if clickout_dict[k] in list(set(v[::-1]))[:3]:
            first_3_count = first_3_count + 1
        if clickout_dict[k] in list(set(v[::-1]))[:4]:
            first_4_count = first_4_count + 1
        if clickout_dict[k] in list(set(v[::-1]))[:5]:
            first_5_count = first_5_count + 1
            
print('Total sessions: ' + str(count_sessions))
print('Picked last hotel: ' + str(first_count))
print('Picked among last 2 hotels ' + str(first_2_count))
print('Picked among last 3 hotels ' + str(first_3_count))
print('Picked among last 4 hotels ' + str(first_4_count))
print('Picked among last 5 hotels ' + str(first_5_count))

Total sessions: 649297
Picked last hotel: 406735
Picked among last 2 hotels 542818
Picked among last 3 hotels 592924
Picked among last 4 hotels 616018
Picked among last 5 hotels 627857


In [11]:
session_dict

{'000056cd97ce2': ['2784325', '3811810', '3811810', '3811810'],
 '000066611146f': ['5479306'],
 '00008c1f39c46': ['3501822'],
 '0000be39860d7': ['445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '445081',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '76308',
  '76308',
  '76308',
  '76308',
  '76308',
  '76308',
  '76308',
  '76308',
  '4920008',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237',
  '79237'],
 '0000c0fb51e11': ['673981', '673981', '673981', '10369176', '10369176'],
 '0000c8490c155': ['42757', '6626164'],
 '0000d4284ff9b': ['1179940', '4590810', '4590810'],
 '00011005327e1': ['1216256',
  '1216256',
  '1216256',
  '1216256',
  '1216256',
  '1216256',
